In [1]:
%load_ext autoreload
%autoreload 2

In [62]:
import os

from dotenv import load_dotenv
load_dotenv()

from llama_index.core import (
    SimpleDirectoryReader, 
    VectorStoreIndex, 
    load_index_from_storage,
    StorageContext,
)
from llama_parse import LlamaParse
from llama_index.core import Settings
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.query_pipeline import QueryPipeline as QP, InputComponent, FnComponent
from llama_index.core.tools import QueryEngineTool, ToolMetadata

from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.anthropic import Anthropic
from llama_index.core.node_parser import SentenceSplitter
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.objects import ObjectIndex
from llama_index.packs.agents_coa import CoAAgentPack
from llama_index.tools.code_interpreter.base import CodeInterpreterToolSpec# from llama_index.core.agent import FunctionCallingAgent
from llama_index.core.agent import ReActAgent, StructuredPlannerAgent, FunctionCallingAgentWorker

import dspy
from dspy.predict.llamaindex import DSPyComponent, LlamaIndexModule

In [2]:
turbo = dspy.OpenAI(model='gpt-4o')
dspy.settings.configure(lm=turbo)

/Users/loyalshababo/dev/rtfm/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [46]:
class GenerateAnswer(dspy.Signature):
    """Answer questions about formatting USB commands to the DLPC900."""

    context = dspy.InputField(desc="Relevant sections of the DLPC900 Programmer's guide.")
    question = dspy.InputField()
    answer = dspy.OutputField(desc=f"""A command to communicate with the device over USB. \
                              The command should be returned as a Python list of bit strings. \
                              
                              Be sure to include and label all parts of the command including the header as specified in Section 1.2 of the Programmer's guide.""")
    
    #                               The `response` you provide will be taken as given as put directly used as follows `dev.write(end_point, response)`. \

In [8]:
# set up parser
parser = LlamaParse(
    result_type="markdown",  # "markdown" and "text" are available
    use_vendor_multimodal_model=True,
    vendor_multimodal_model_name="anthropic-sonnet-3.5"
)

In [9]:
node_parser = SentenceSplitter()

dlpc9000_docs_dirpath = "../../manuals/dmd"
persist_dir_root = "./storage/dlpc9000"
doc_name = 'dlpu018j'
doc_path = os.path.join(dlpc9000_docs_dirpath, "*.pdf")
if not os.path.exists(persist_dir_root):
    os.makedirs(persist_dir_root)



print(f"Building agent for document {doc_name}")


doc_persist_dir = os.path.join(persist_dir_root, doc_name)
if not os.path.exists(doc_persist_dir):
    # build vector index
    file_extractor = {".pdf": parser, ".html": parser}
    doc = SimpleDirectoryReader(input_files=[doc_path], file_extractor=file_extractor).load_data()
    # vector_index = VectorStoreIndex(doc)
    nodes = node_parser.get_nodes_from_documents(doc)
    vector_index = VectorStoreIndex(nodes)
    vector_index.storage_context.persist(
        persist_dir=doc_persist_dir
    )
else:
    vector_index = load_index_from_storage(
        StorageContext.from_defaults(persist_dir=doc_persist_dir),
    )

Building agent for document dlpu018j


In [64]:
Settings.llm = Anthropic(model='claude-3-5-sonnet-20240620')
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

In [65]:
vector_query_engine = vector_index.as_query_engine(llm=Settings.llm, similarity_top_k=10)
# summary_query_engine = summary_index.as_query_engine(llm=Settings.llm)

# define tools
query_engine_tools = [
    QueryEngineTool(
        query_engine=vector_query_engine,
        metadata=ToolMetadata(
            name="vector_tool",
            description="This tool can query the DLPC900 (aka DLPC9000EVM) Programmer's Guide which contains information about communicating with the board and specifyihng.",
        ),
    ),
    # QueryEngineTool(
    #     query_engine=summary_query_engine,
    #     metadata=ToolMetadata(
    #         name="summary_tool",
    #         description=summary_query_engine.query("Summarize this document so a tool built upon it knows how to use it.").response,
    #     ),
    # ),
]

# build agent
function_llm = Anthropic(model="claude-3-5-sonnet-20240620")
agent = FunctionCallingAgentWorker.from_tools(
    query_engine_tools + CodeInterpreterToolSpec().to_tool_list(),
    llm=function_llm,
    verbose=True,
    # system_prompt=f"""\
    #             This tools has access to information about communicating with the DLPC900 (aka DLPC9000EVM) board over serial USB. \
    #             It can go through this document and understand how to format read and write commands to the board. \
    #             It can answer questions about how to format serial commands for different board commands. \
    #             It can can also answer questions going in the other direction, given a board command, it will provide the serial command. \
    #             Remember to always reference Section 1.2 when giving an answer since this explains how to format USB commands. \
    #             When you give an answer, please report what tables you used to get information. \
    #             Be careful, it also contains I2C commands in the same tables. \
    #             """,
)

In [47]:
class DLPC900CommandFormatter(dspy.Module):
    def __init__(self, dlpc_programmer_manual_agent):
        super().__init__()
        self.manual_agent = dlpc_programmer_manual_agent
        self.generate_answer = dspy.Predict(GenerateAnswer)


    def forward(self, question):
        response = self.query_engine.query(question)
        context = response.response
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

In [48]:
rag_module = RAG(vector_index.as_query_engine(similarity_top_k=5))

In [49]:
question = "What command should I send over USB to turn idle mode off?"
pred = rag_module(question)
print(f"Question: {question}")
print(f"Predicted Answer: {pred.answer}")

Question: What command should I send over USB to turn idle mode off?
Predicted Answer: Context: To turn off Idle Mode, you should send the command 0x0201 over USB.
Question: What command should I send over USB to turn idle mode off?
Answer: A command to communicate with the device over USB. Be sure to include and label all parts of the command including the header as specified in Section 1.2 of the Programmer's guide.

Header:
- Command: 0x02 (2 bytes)
- Sub-command: 0x01 (2 bytes)

Complete USB Command:
```
0x02 0x01
```


In [50]:
dspy_module = dspy.ChainOfThought(GenerateAnswer)

In [51]:
question = "What command should I send over USB to turn idle mode off?"
pred = rag_module(question)
print(f"Question: {question}")
print(f"Predicted Answer: {pred.answer}")

Question: What command should I send over USB to turn idle mode off?
Predicted Answer: Context: To turn off Idle Mode, you should send the command 0x8D over USB.
Question: What command should I send over USB to turn idle mode off?
Answer: A command to communicate with the device over USB. Be sure to include and label all parts of the command including the header as specified in Section 1.2 of the Programmer's guide.

Header:
- Command: 0x1A (assuming 0x1A is the command header for sending commands)
- Length: 0x01 (length of the payload, which is 1 byte in this case)
- Reserved: 0x00 (reserved byte, typically set to 0)

Payload:
- Command: 0x8D


In [16]:
dspy_module = dspy.ChainOfThought(GenerateAnswer)

retriever_post = FnComponent(
    lambda contexts: "\n\n".join([n.get_content() for n in contexts])
)

p = QP(verbose=True)
p.add_modules(
    {
        "input": InputComponent(),
        "retriever": retriever,
        "retriever_post": retriever_post,
        "synthesizer": dspy_module,
    }
)
p.add_link("input", "retriever")
p.add_link("retriever", "retriever_post")
p.add_link("input", "synthesizer", dest_key="query_str")
p.add_link("retriever_post", "synthesizer", dest_key="context_str")


dspy_qp = LlamaIndexModule(p)

AttributeError: 'ChainOfThought' object has no attribute 'sub_query_components'

In [14]:
output = dspy_qp(query_str="What command should I send over USB to turn idle mode off?")

> Running module input with input: 
query_str: What command should I send over USB to turn idle mode off?

> Running module retriever with input: 
input: What command should I send over USB to turn idle mode off?

> Running module retriever_post with input: 
contexts: [NodeWithScore(node=TextNode(id_='84e94928-1c22-4e0d-a43b-157276a7f46e', embedding=None, metadata={'file_path': '../../manuals/dmd/dlpu018j.pdf', 'file_name': 'dlpu018j.pdf', 'file_type': 'application...

> Running module synthesizer with input: 
query_str: What command should I send over USB to turn idle mode off?
context_str: ## 2.3.1.2 DMD Standby and Idle Modes

When a DMD is idle and not actively projecting data, enable DMD Idle Mode or Standby Mode to assist in maximizing DMD lifetime.

Enable Idle Mode during shorter ...



In [ ]:
from dspy import Example

train_examples = [
    Example(query_str="What command should I send over USB to turn idle mode off?", answer="0x00 0x00"),
    Example(query_str="What did the author do during his time at YC?", answer="organizing a Summer Founders Program, funding startups, writing essays, working on a new version of Arc, creating Hacker News, and developing internal software for YC")
]

train_examples = [t.with_inputs("query_str") for t in train_examples]